In [ ]:
import os
os.environ["NGSIM_DIR"] = "/home/surender/Downloads/NGSIM"
os.environ["OPENDD_DIR"] = "/home/surender/Downloads/openDD"
os.environ["CARLA_PATH"] = "/home/surender/Downloads/carlaOld"
import sys
#sys.path.append('/home/surender/Downloads/CARLA_0.9.9.4/PythonAPI/carla/dist')
import carla
import random
import argparse

from carla_real_traffic_scenarios.carla_maps import CarlaMaps
from carla_real_traffic_scenarios.ngsim import NGSimDatasets, DatasetMode
from carla_real_traffic_scenarios.ngsim.scenario import NGSimLaneChangeScenario
from carla_real_traffic_scenarios.opendd.scenario import OpenDDScenario
from carla_real_traffic_scenarios.reward import RewardType
from carla_real_traffic_scenarios.scenario import Scenario

from carla_birdeye_view import BirdViewProducer, BirdViewCropType, PixelDimensions
from PIL import Image
from IPython.display import clear_output, Image, display, HTML
import cv2

%matplotlib tk
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import threading
import time



In [ ]:
from src.VehicleActor import *

In [ ]:
def cmd_carla1():
    os.system("/home/surender/Downloads/carlaOld/CarlaUE4.sh -benchmark -fps=10 -quality-level=Low -opengl -Resx=300 -Resy=300 -NoVSync")

In [ ]:
def cmd_carla():
    os.system("DISPLAY= /home/surender/Downloads/carlaOld/CarlaUE4.sh -benchmark -fps=10 -quality-level=Low -opengl -Resx=300 -Resy=300 -NoVSync")

In [ ]:
'''
def parser_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", choices=["ngsim", "opendd"], default="opendd")
    parser.add_argument("--host", default="127.0.0.1")
    parser.add_argument("--port", default=2000, type=int)
    parser.add_argument("--num-episodes", default=10, type=int)
    args = parser.parse_args()
    return args

'''

def prepare_ngsim_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("NGSIM_DIR")
    #data_dir = os.listdir('/home/surender/Downloads/NGSIM')
    assert data_dir, "Path to the directory with NGSIM dataset is required"
    ngsim_map = NGSimDatasets.list()
    ngsim_dataset = ngsim_map[1]
    client.load_world(ngsim_dataset.carla_map.level_path)
    return NGSimLaneChangeScenario(
        ngsim_dataset,
        dataset_mode=DatasetMode.TRAIN,
        data_dir=data_dir,
        reward_type=RewardType.DENSE,
        client=client,
    )

'''
def prepare_opendd_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("OPENDD_DIR")
    assert data_dir, "Path to the directory with openDD dataset is required"
    maps = ["rdb1", "rdb2", "rdb3", "rdb4", "rdb5", "rdb6", "rdb7"]
    map_name = random.choice(maps)
    carla_map = getattr(CarlaMaps, map_name.upper())
    client.load_world(carla_map.level_path)
    return OpenDDScenario(
        client,
        dataset_dir=data_dir,
        dataset_mode=DatasetMode.TRAIN,
        reward_type=RewardType.DENSE,
        place_name=map_name,
    )

'''
def prepare_ego_vehicle(world: carla.World) -> carla.Actor:
    car_blueprint = world.get_blueprint_library().find("vehicle.audi.a2")

    # This will allow external scripts like manual_control.py or no_rendering_mode.py
    # from the official CARLA examples to take control over the ego agent
    car_blueprint.set_attribute("role_name", "hero")

    # spawn points doesnt matter - scenario sets up position in reset
    ego_vehicle = world.spawn_actor(
        car_blueprint, carla.Transform(carla.Location(0, 0, 500), carla.Rotation())
    )

    assert ego_vehicle is not None, "Ego vehicle could not be spawned"

    # Setup any car sensors you like, collect observations and then use them as input to your model
    return ego_vehicle

In [ ]:
p = threading.Thread(target = cmd_carla)
p.start()


In [ ]:
host = "localhost"
port = 2000
client = carla.Client(host,port)

In [ ]:
scenario = prepare_ngsim_scenario(client)

In [ ]:
world = client.get_world()
spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)




In [ ]:
world.tick()

In [ ]:
scenario.reset(ego_vehicle)

In [ ]:
manager = VehicleManager(ego_vehicle,world,scenario)

In [ ]:
world.tick()

In [ ]:
manager.update_neighbours()

In [ ]:
manager.neighbour_profiles

In [ ]:
manager.update_buffers()

In [ ]:
e,n = manager.return_data()

In [ ]:
world.tick()


In [ ]:
manager.vehicleSensorslist[1764].current_frame

In [ ]:
plt.imshow(e[0])

In [ ]:
plt.imshow(e[1])

In [ ]:
plt.imshow(n[0][1])

In [ ]:
manager.update_neighbours()
world.tick()
time.sleep(1)
manager.update_buffers()
ego,n = manager.return_data()

In [ ]:
manager.update_buffers()

In [ ]:
ego,n = manager.return_data()

In [ ]:
ego

In [ ]:
n

In [ ]:
sys.getsizeof(n)

In [ ]:
n[0][0].shape

In [ ]:
sys.getsizeof(e)

In [ ]:
manager.vehicleSensorslist[0].start_sensors()

In [ ]:
manager.vehicleSensorslist[0].gnss_buffer

In [ ]:
scenario.step(ego_vehicle).chauffeur_cmd = 1

In [ ]:
scenario._lane_change

In [ ]:
vehicles[0]._vehicle_by_vehicle_id

In [ ]:
scenario._

In [ ]:
ego_vehicle.get_transform().location.z

In [ ]:
l1 = list(scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id.keys())

In [ ]:
vehicles = scenario._ngsim_recording.step()

In [ ]:
l3 = [x.id for x in vehicles]

In [ ]:
scenario

In [ ]:
len(d.keys())

In [ ]:
len(l3)

In [ ]:
world.tick()

In [ ]:
#scenario.step(ego_vehicle)

In [ ]:
d = scenario._ngsim_vehicles_in_carla._vehicle_by_vehicle_id

In [ ]:
dist = {}
for i in d:
    c = ego_vehicle.get_location().distance(d[i].get_location())
    dist[i] = c

In [ ]:
updated_dist = dict(sorted(dist.items(), key=lambda item: item[1]))

In [ ]:
list(updated_dist.keys())[:4]

In [ ]:
dist

In [ ]:
d

In [ ]:
d

In [ ]:
d[ego_vehicle.id]

In [ ]:
world.tick()

In [ ]:
ego_vehicle.get_location().distance(d[48].get_location())

In [ ]:
min_d = 10000
min_ind = -1
for i in l1:
    dist = ego_vehicle.get_location().distance(d[i].get_location())
    print(dist)
    if dist<min_d:
        min_d = dist
        min_ind = i

In [ ]:
min_ind

In [ ]:
min_d

In [ ]:
d[146].get_location().z

In [ ]:
ego_vehicle.get_location().z

In [ ]:
d[48].get_location().z

In [ ]:
(d[48].get_location() + ego_vehicle.get_location()).z

In [ ]:
d[236].get_location().x

In [ ]:
l1

In [ ]:
list(world.get_actors())

In [ ]:
ego_vehicle.get_location().z

In [ ]:
c  =vehicles[0].transform.position

In [ ]:
c.x

In [ ]:
ego_vehicle.id

In [ ]:
len(l3)

In [ ]:
l1

In [ ]:
scenario.reset(ego_vehicle)

In [ ]:
for i in range(len(l1)):
    if l1[i] == l3[i]:
        continue
    else:
        print(l1[i],l3[i],i)

In [ ]:
vehicles[0].transform == ego_vehicle.get_transform()

In [ ]:
vehicles = scenario._ngsim_recording.step()

In [ ]:

scenario._ngsim_vehicles_in_carla

In [ ]:
world.tick()

In [ ]:
world.get_actors()[1]

In [ ]:
list(world.get_actors())

In [ ]:


birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=150, height=336),
    pixels_per_meter=4,
    crop_type=BirdViewCropType.FRONT_AND_REAR_AREA
)

In [ ]:
world.tick()

In [ ]:
def print_data(dat):
    print(dat.latitude,dat.longitude, dat.altitude)

In [ ]:
def prit_data(dat):
    print(dat.frame)
    points = np.frombuffer(dat.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (len(dat), 4))
    print(points)

In [ ]:
def check_img(img):
    print(img.frame)
    array = np.frombuffer(img.raw_data, dtype=np.dtype("uint8")) 
    array = np.reshape(array, (img.height, img.width, 4)) # RGBA format
    array = array[:, :, :3] #  Take only RGB
    
    
    img = Image.fromarray(array)
    img = img.resize((299, 299), Image.ANTIALIAS)
    input_data = np.array(img)

In [ ]:
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(320))
cam_bp.set_attribute("image_size_y",str(320))
cam_bp.set_attribute("fov",str(150))
cam_location = carla.Location(2,0,1)
cam_rotation = carla.Rotation(0,0,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
rgb_front_listener = ego_cam
rgb_front_listener.listen(lambda image: check_img(image))

In [ ]:
ldr_bp = world.get_blueprint_library().find('sensor.camera.rgb')
ldr_loc = carla.Location(0,0,0)
ldr_rot = carla.Rotation(0,0,0)
ldr_bp.set_attribute("range",'500.0')


ldr_transform = carla.Transform(ldr_loc,ldr_rot)
ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_ldr.listen(lambda dat: prit_data(dat))

In [ ]:
ldr_transform.location.z

In [ ]:
list(world.get_actors())

In [ ]:
world.tick()

In [ ]:
for i in d:
    print(d[i].id,i)

In [ ]:
del ego_sadr

In [ ]:
ego_sadr

In [ ]:
world.tick()

In [ ]:
sadr_bp = world.get_blueprint_library().find('sensor.other.gnss')
sadr_loc = carla.Location(0,0,0)
sadr_rot = carla.Rotation(0,0,0)
#ldr_bp.set_attribute("range",'10.0')


sadr_transform = carla.Transform(sadr_loc,sadr_rot)
ego_sadr = world.spawn_actor(sadr_bp,sadr_transform, attach_to=d[1036], attachment_type=carla.AttachmentType.Rigid)
ego_sadr.listen(lambda dat: print_data(dat))

In [ ]:
sdr_bp = world.get_blueprint_library().find('sensor.other.gnss')
sdr_loc = carla.Location(0,0,0)
sdr_rot = carla.Rotation(0,0,0)
#ldr_bp.set_attribute("range",'10.0')


sdr_transform = carla.Transform(sdr_loc,sdr_rot)
ego_sdr = world.spawn_actor(sdr_bp,sdr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_sdr.listen(lambda dat: print_data(dat))

In [ ]:
world.tick()

In [ ]:
sensor.other.imu

In [ ]:
def reload_carla():
    global host,port,client,scenario, world,spectator,ego_vehicle,birdview_producers,p
    p = threading.Thread(target = cmd_carla)
    p.start()
    print("Restarting carla.....")
    time.sleep(5)
    
    
    print("Connecting to carla...")
    host = "localhost"
    port = 2000
    client = carla.Client(host,port)
    scenario = prepare_ngsim_scenario(client)
    world = client.get_world()
    spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=150, height=336),
    pixels_per_meter=4,
    crop_type=BirdViewCropType.FRONT_AND_REAR_AREA
    )
    ldr_bp = world.get_blueprint_library().find('sensor.other.radar')
    ldr_loc = carla.Location(0,0,0)
    ldr_rot = carla.Rotation(0,0,0)
    ldr_bp.set_attribute("range",'10.0')


    ldr_transform = carla.Transform(ldr_loc,ldr_rot)
    ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
    ego_ldr.listen(lambda dat: load_radar(dat))

In [ ]:
class Hp():
    
    def __init__(self):
        self.nb_steps = 1500
        self.episode_length = 1000
        self.learning_rate = 0.02
        self.nb_directions = 16
        self.nb_best_directions = 16
        assert self.nb_best_directions <= self.nb_directions
        self.noise = 0.03
        self.seed = 10
        #self.env_name = 'HalfCheetahBulletEnv-v0'

# Normalizing the states


In [ ]:
hp = Hp()
np.random.seed(hp.seed)
state_limit = 15


input_state = [0]*state_limit*4
nb_inputs = state_limit * 4
nb_outputs = 2
save_every = 25

render = False

In [ ]:
reward_vector = []

In [ ]:

class Normalizer():
    
    def __init__(self, nb_inputs):
        global hp
        self.n = np.zeros(nb_inputs)
        self.mean = np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)
    
    def observe(self, x):
        global hp
        self.n += 1.
        last_mean = self.mean.copy()
        
        self.mean += (x - self.mean) / self.n
        
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff / self.n).clip(min = 1e-2)
    
    def normalize(self, inputs):
        global hp
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        if np.sum(obs_std) == 0:
            print("TILTED")
        return (inputs - obs_mean) / obs_std

# Building the AI

class Policy():
    
    def __init__(self, input_size, output_size):
        
        self.theta = np.zeros((output_size, input_size))
    
    def evaluate(self, input, delta = None, direction = None):
        global hp
        if direction is None:
            return self.theta.dot(input)
        elif direction == "positive":
            return (self.theta + hp.noise*delta).dot(input)
        else:
            return (self.theta - hp.noise*delta).dot(input)
    
    def sample_deltas(self):
        global hp
        return [np.random.randn(*self.theta.shape) for _ in range(hp.nb_directions)]
    
    def update(self, rollouts, sigma_r):
        global hp
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, d in rollouts:
            step += (r_pos - r_neg) * d
        if np.sum(step) == 0:
            print("STEPS ZERO")
            for r_pos, r_neg, d in rollouts:
                step = np.ones(self.theta.shape) * 0.05 * d
            
        if sigma_r == 0:
            print("NO ZERO")
            sigma_r = 0.01
        self.theta += hp.learning_rate / (hp.nb_best_directions * sigma_r) * step
        
    def save_policy(self,step_num):
        np.savetxt("Policy_" + str(step_num) +".gz", self.theta)
    
    def load_policy(self,step_num):
        self.theta = np.loadtxt("Policy_" + str(step_num) +".gz")

# Exploring the policy on one specific direction and over one episode

def explore(env, normalizer, policy, direction = None, delta = None):
    global ego_vehicle,input_state,render,hp
    env.reset(ego_vehicle)
    world.tick()
    state = input_state
    done = False
    num_plays = 0.
    sum_rewards = 0
    while not done:
        normalizer.observe(state)
        state = normalizer.normalize(state)
        action = policy.evaluate(state, delta, direction)
        action[0] = np.clip(action[0],0.0,1.0) # Throttle
        action[1] = np.clip(action[1],-1.0,1.0) # Steering
        #action[2] = np.clip(action[2],0.0,1.0) #Brake
        
        if render:
            birdview = birdview_producer.produce(
            agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
            )
            rgb = BirdViewProducer.as_rgb(birdview)
            cv2.imshow('Frame',rgb)
            if cv2.waitKey(25) & 0xFF == ord('q'):

                  break
        
        ego_vehicle.apply_control(carla.VehicleControl(throttle=action[0], steer=action[1]))
        cmd, reward, done, _ = env.step(ego_vehicle)
        #reward = np.clip(reward,0,1)
        #reward = max(reward, -1)
        #reward = max(min(reward, 1), -1)
        world.tick()
        sum_rewards += reward
        num_plays += 1
    cv2.destroyAllWindows()
    return sum_rewards


def train(env, policy, normalizer, hp, resume=None, resume_step = 0):
    global render,save_every,reward_vector,scenario
    max_reward =  -1
    if resume == True:
        policy.load_policy(resume_step)
        
    for step in range(resume_step, hp.nb_steps):
        try:
            if step > 1000:
                render = True
            # Initializing the perturbations deltas and the positive/negative rewards
            deltas = policy.sample_deltas()
            positive_rewards = [0] * hp.nb_directions
            negative_rewards = [0] * hp.nb_directions

            # Getting the positive rewards in the positive directions
            for k in range(hp.nb_directions):
                positive_rewards[k] = explore(env, normalizer, policy, direction = "positive", delta = deltas[k])

            # Getting the negative rewards in the negative/opposite directions
            for k in range(hp.nb_directions):
                negative_rewards[k] = explore(env, normalizer, policy, direction = "negative", delta = deltas[k])

            # Gathering all the positive/negative rewards to compute the standard deviation of these rewards
            all_rewards = np.array(positive_rewards + negative_rewards)
            sigma_r = all_rewards.std()

            # Sorting the rollouts by the max(r_pos, r_neg) and selecting the best directions
            scores = {k:max(r_pos, r_neg) for k,(r_pos,r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
            order = sorted(scores.keys(), key = lambda x:scores[x], reverse = True)[:hp.nb_best_directions]
            rollouts = [(positive_rewards[k], negative_rewards[k], deltas[k]) for k in order]

            # Updating our policy
            policy.update(rollouts, sigma_r)

            if step%save_every == 0:
                policy.save_policy(step)
            

            # Printing the final reward of the policy after the update
            reward_evaluation = explore(env, normalizer, policy)
            
            if reward_evaluation > max_reward:
                policy.save_policy(step)
                print("Max reward:",reward_evaluation)
                max_reward = reward_evaluation
            print('Step:', step, 'Reward:', reward_evaluation)
            reward_vector.append(reward_evaluation)
        except Exception as e:
            print(e)
            reload_carla()
            time.sleep(5)
            reward_vector.append(reward_vector[-1])
            env = scenario
            print("Carla Reloaded and connected, continue training!")
            continue

In [ ]:
def load_radar(dat):
    global input_state,state_limit
    points = np.frombuffer(dat.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (len(dat), 4))
    radar_Data = points[np.argsort(points[:, -1])]
        
    state = radar_Data[:state_limit].flatten()
    state.resize(state_limit*4,refcheck = False)
    input_state =  state

In [ ]:
ldr_bp = world.get_blueprint_library().find('sensor.other.radar')
ldr_loc = carla.Location(0,0,0)
ldr_rot = carla.Rotation(0,0,0)
ldr_bp.set_attribute("range",'10.0')


ldr_transform = carla.Transform(ldr_loc,ldr_rot)
ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_ldr.listen(lambda dat: load_radar(dat))

In [ ]:
policy = Policy(nb_inputs, nb_outputs)
normalizer = Normalizer(nb_inputs)


In [ ]:
train(scenario, policy, normalizer, hp)

In [ ]:
sorted_array

In [ ]:
sorted_array.flatten().shape

In [ ]:
a = np.random.rand(10,4).flatten()

In [ ]:
a.resize(50,refcheck=False)

In [ ]:
a.shape